<a href="https://colab.research.google.com/github/EmmaHsueh/PL_project/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日常支出速算與分攤（作業一）
目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。
AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。
Sheet 欄位：date, category, item, amount, payer

https://docs.google.com/spreadsheets/d/1IbbAn-J65AR1CDjqmtNR4IvOmmhwyJoNPg5pAgrrFXI/edit?gid=0#gid=0

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1IbbAn-J65AR1CDjqmtNR4IvOmmhwyJoNPg5pAgrrFXI/edit?gid=0#gid=0')

In [23]:
import datetime
# 打開你的試算表
sheet_name = '工作表1' # 請換成你的試算表名稱
# sheet = gc.open(sheet_name).sheet1 # 移除這行，因為我們已經用 URL 開啟試算表
sheet = gsheets.sheet1 # 使用 gsheets 變數來取得第一個工作表 (通常是 sheet1)

def record_expense():
    """
    記錄一筆日常支出
    """
    # 讓使用者輸入日期
    date_str = input("請輸入日期 (YYYY-MM-DD)：")
    item = input("請輸入品項：")
    amount = float(input("請輸入金額："))
    category = input("請輸入類別：") # 新增類別輸入
    payer = input("請輸入付款方式：") # 新增付款人輸入

    # 建立要寫入的資料列
    # 這裡的順序要對應你的Google Sheets的 date, category, item, amount, payer 欄位
    row_data = [date_str, category, item, amount, payer] # 修改資料順序與新增欄位

    # 將資料寫入Google Sheets
    sheet.append_row(row_data)

    print(f"成功記錄：日期={date_str}, 類別={category}, 品項={item}, 金額={amount}, 付款方式={payer}")

# 執行記錄功能
record_expense() # 先註解掉這行，避免重複執行

# Read all data from the sheet
data = gsheets.sheet1.get_all_values()

# Convert the data to a pandas DataFrame, using the first row as headers
# Assuming the first row in the sheet contains the headers: date, category, item, amount, payer
df_expenses = pd.DataFrame(data[1:], columns=data[0])

# Display the first few rows of the DataFrame
display(df_expenses.head())

請輸入日期 (YYYY-MM-DD)：2025-09-11
請輸入品項：dinner
請輸入金額：130
請輸入類別：daily
請輸入付款方式：cash
成功記錄：日期=2025-09-11, 類別=daily, 品項=dinner, 金額=130.0, 付款方式=cash


,日期,時間,品項,金額,
0,2025-09-11,daily,coffee,45,cash
1,2025-09-11,daily,lunch,120,cash
2,2025-09-11,daily,dinner,120,cash
3,2025-09-11,daily,dinner,130,cash


## 讀取資料至 dataframe

### Subtask:
從 Google Sheet 讀取現有資料，並儲存在一個 pandas DataFrame 中。


**Reasoning**:
Read data from the Google Sheet and load it into a pandas DataFrame with specified column names.



## 修改記錄支出函數

### Subtask:
修改 `record_expense` 函數，讓它將新的支出項目添加到現有的 DataFrame 中。


**Reasoning**:
Modify the `record_expense` function to append the new expense data to the `df_expenses` DataFrame instead of writing to the Google Sheet directly.



## 將 dataframe 寫回 google sheet

### Subtask:
將更新後的 DataFrame 寫回 Google Sheet 中。


**Reasoning**:
Clear the existing data in the Google Sheet and then write the updated DataFrame, including the header, back to the sheet.



## Summary:

### Data Analysis Key Findings

*   The existing data was successfully read from the Google Sheet into a pandas DataFrame (`df_expenses`).
*   The `record_expense` function was modified to append new expense data directly to the `df_expenses` DataFrame using `pd.concat`.
*   The updated `df_expenses` DataFrame was successfully written back to the Google Sheet after clearing the existing content.

### Insights or Next Steps

*   The current process overwrites the entire sheet each time data is written. Consider implementing a method to append only new rows to avoid unnecessary data transfer and potential issues with concurrent updates.
*   Add error handling for user input in the `record_expense` function (e.g., ensuring amount is a number, date format is correct).


In [26]:
def calculate_share():
    """
    讀取Google Sheets數據並計算分攤
    """
    # 讀取所有數據
    data = sheet.get_all_records()

    if not data:
        print("沒有任何支出記錄。")
        return

    # 計算總金額
    total_amount = sum(row['金額'] for row in data)

    # 讓使用者輸入分攤人數
    while True:
        try:
            num_people_str = input("請輸入分攤人數：")
            num_people = int(num_people_str)
            if num_people <= 0:
                print("分攤人數必須大於零。")
            else:
                break
        except ValueError:
            print("請輸入有效的數字。")

    if num_people > 0:
        share_per_person = total_amount / num_people
        print("\n--- 支出分攤速算 ---")
        print(f"總支出金額：{total_amount:.2f} 元")
        print(f"每人應分攤金額（假設 {num_people} 人）：{share_per_person:.2f} 元")
    else:
        print("無法計算分攤金額，因為分攤人數為零。")

# 執行分攤計算
calculate_share()

請輸入分攤人數：ˇ
請輸入有效的數字。
請輸入分攤人數：3

--- 支出分攤速算 ---
總支出金額：415.00 元
每人應分攤金額（假設 3 人）：138.33 元


In [29]:
def main():
    while True:
        print("\n--- 支出記錄與分攤程式 ---")
        print("1. 記錄一筆支出")
        print("2. 計算支出總額與分攤")
        print("3. 退出")

        choice = input("請選擇功能（1/2/3）：")

        if choice == '1':
            record_expense()

            data = gsheets.sheet1.get_all_values()
            df_expenses = pd.DataFrame(data[1:], columns=data[0])
            display(df_expenses.head())
        elif choice == '2':
            calculate_share()
        elif choice == '3':
            print("程式已退出。")
            break
        else:
            print("無效的選擇，請重新輸入。")

# 運行主程式
main()


--- 支出記錄與分攤程式 ---
1. 記錄一筆支出
2. 計算支出總額與分攤
3. 退出
請選擇功能（1/2/3）：1
請輸入日期 (YYYY-MM-DD)：2025-09-12
請輸入品項：lunch
請輸入金額：120
請輸入類別：daily
請輸入付款方式：cash
成功記錄：日期=2025-09-12, 類別=daily, 品項=lunch, 金額=120.0, 付款方式=cash


,日期,時間,品項,金額,
0,2025-09-11,daily,coffee,45,cash
1,2025-09-11,daily,lunch,120,cash
2,2025-09-11,daily,dinner,120,cash
3,2025-09-11,daily,dinner,130,cash
4,2025-09-11,daily,coffee,45,cash



--- 支出記錄與分攤程式 ---
1. 記錄一筆支出
2. 計算支出總額與分攤
3. 退出
請選擇功能（1/2/3）：2
請輸入分攤人數：3

--- 支出分攤速算 ---
總支出金額：580.00 元
每人應分攤金額（假設 3 人）：193.33 元

--- 支出記錄與分攤程式 ---
1. 記錄一筆支出
2. 計算支出總額與分攤
3. 退出
請選擇功能（1/2/3）：3
程式已退出。
